In [18]:
import pandas as pd
import numpy as np
import joblib
import json
import time
import collections
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
pd.__version__

'0.24.2'

In [36]:
df = pd.read_csv('raw_data/TenantInfo-usage-and-verbatim_shuffled_inf.csv'#, nrows=1000
                )

In [37]:
df.shape

(161345, 230)

In [29]:
df.head()

,TenantId,Comment,Theme,CreatedDate,CreateDateOfFirstSubscription,CountryCode,AllupSeats,EXOSubscriptionsCount,OD4BSubscriptionsCount,SfBSubscriptionsCount,TeamsSubscriptionsCount,PaidCount,ProjectSubscriptionsCount,SPOSubscriptionsCount,ActivatedSubscriptionTotalCount,VisioSubscriptionsCount,TrialSubscriptionsCount,NonTrialSubscriptionsCount,Languange,DataCenterInstance,DataCenterModel,HasEXO,HasSPO,HasOD4B,HasSfB,HasYammer,HasTeams,HasTeamsFreemium,HasKaizala,HasProPlus,HasAADP,HasAIP,HasAATP,HasIntune,HasMCAS,HasWDATP,HasAudioConference,HasPhoneSystem,HasEdiscovery,HasCompliance,HasThreatIntelligence,HasCustomerLockbox,HasOATP,HasAADPP2,HasAIPP2,HasWindows,HasO365CAS,HasCASDiscovery,HasPAM,HasPowerBI,HasPowerBIPremium,HasPowerBIPro,HasVisio,HasProject,HasNonTrial,HasSubscription_AllCounted,WithoutTenantAdmin,OrgNameEnteredInCommerce,HasPartnerTenants,SignupLocationInfo_Country,SignupLocationInfo_CountryCode,SignupLocationInfo_Region,Subscriptions_HasPaidSeats,TopParents_AreaName,TopParents_BigAreaName,TopParents_CountryCode,TopParents_Industry,TopParents_RegionName,TopParents_SegmentGroup,TopParents_SubRegionName,TopParents_VerticalName,EXOEnabledUsers,SPOEnabledUsers,OD4BEnabledUsers,SFBEnabledUsers,TeamEnabledUsers,YammerEnabledUsers,PPDEnabledUsers,KaizalaEnabledUsers,AADPEnabledUsers,AIPEnabledUsers,AATPEnabledUsers,IntuneEnabledUsers,MCASEnabledUsers,WDATPEnabledUsers,AudioConferenceEnabledUsers,PhoneSystemEnabledUsers,EdiscoveryEnabledUsers,ComplianceEnabledUsers,ThreatIntelligenceEnabledUsers,CustomerLockboxEnabledUsers,OATPEnabledUsers,AADPP2EnabledUsers,AIPP2EnabledUsers,WindowsEnabledUsers,O365CASEnabledUsers,CASDiscoveryEnabledUsers,PAMEnabledUsers,O365EnabledUsers,EMSEnabledUsers,M365EnabledUsers,O365E5EnabledUsers,EMSE5EnabledUsers,M365E5EnabledUsers,TotalUsers,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidYammerSeats,PaidTeamsSeats,PaidSFBSeats,PaidKaizalaSeats,PaidProplusSeats,PaidAADPSeats,PaidAIPSeats,PaidAATPSeats,PaidIntuneSeats,PaidMCASSeats,PaidWDATPSeats,PaidAudioConferenceSeats,PaidPhoneSystemSeats,PaidOATPSeats,PaidAADPP2Seats,PaidAIPP2Seats,PaidWindowsSeats,PaidO365CASSeats,PaidCASDiscoverySeats,PaidPAMSeats,PaidPowerBISeats,PaidPowerBIPremiumSeats,PaidPowerBIProSeats,PaidEMSSeats,PaidM365Seats,PaidOfficeSeats,HasUsGovCloudOffer,FirstPaidEXOStartDate,FirstPaidSPOStartDate,FirstPaidOD4BStartDate,FirstPaidSfBStartDate,FirstPaidYammerStartDate,FirstPaidTeamsStartDate,FirstPaidProPlusStartDate,FirstPaidAADPStartDate,FirstPaidAIPStartDate,FirstPaidAATPStartDate,FirstPaidIntuneStartDate,FirstPaidMCASStartDate,FirstPaidO365E5SkuStartDate,FirstPaidM365E5SkuStartDate,FirstPaidEMSE5SkuStartDate,Has1YearFreeDomain,Age,AU_exchange_04,AU_sharepoint_04,AU_skype_04,AU_teams_04,AU_od4b_04,AU_onenote_04,AU_word_04,AU_excel_04,AU_powerpoint_04,AU_outlook_04,AU_oatp_04,AU_odsp_04,AU_eslt_04,AU_officeclient_04,AU_exchange_05,AU_sharepoint_05,AU_skype_05,AU_teams_05,AU_od4b_05,AU_onenote_05,AU_word_05,AU_excel_05,AU_powerpoint_05,AU_outlook_05,AU_oatp_05,AU_odsp_05,AU_eslt_05,AU_officeclient_05,AU_exchange,AU_sharepoint,AU_skype,AU_teams,AU_od4b,AU_onenote,AU_word,AU_excel,AU_powerpoint,AU_outlook,AU_oatp,AU_odsp,AU_eslt,AU_officeclient,AR_exchange_04,AR_sharepoint_04,AR_skype_04,AR_teams_04,AR_od4b_04,AR_onenote_04,AR_word_04,AR_excel_04,AR_powerpoint_04,AR_outlook_04,AR_eslt_04,AR_officeclient_04,AR_exchange_05,AR_sharepoint_05,AR_skype_05,AR_teams_05,AR_od4b_05,AR_onenote_05,AR_word_05,AR_excel_05,AR_powerpoint_05,AR_outlook_05,AR_eslt_05,AR_officeclient_05,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06
0,6B0AC2EB-3AE7-46AC-92D8-B5789477B631,Excelente seguimiento y atención. Todas mis du...,"Teams - Download, Setup and Use Microsoft Team...",7/6/2016 7:43:18 AM,7/13/2016 12:00:00 AM,MX,145,0,1,0,0,145,0,0,3,0,0,3,es,Global|GoLocal,PublicCloud|GoLocal,False,False,True,False,False,False,False,False,True,F

In [8]:
with open('raw_data/configure_wText_woPreviousUsage_NotPredectAhead.json', 'r') as f:
    metadata = json.load(f)

metadata

{'input_bool': ['HasEXO',
  'HasSPO',
  'HasOD4B',
  'HasSfB',
  'HasYammer',
  'HasTeams',
  'HasTeamsFreemium',
  'HasKaizala',
  'HasProPlus',
  'HasAADP',
  'HasAIP',
  'HasAATP',
  'HasIntune',
  'HasMCAS',
  'HasWDATP',
  'HasAudioConference',
  'HasPhoneSystem',
  'HasEdiscovery',
  'HasCompliance',
  'HasThreatIntelligence',
  'HasCustomerLockbox',
  'HasOATP',
  'HasAADPP2',
  'HasAIPP2',
  'HasWindows',
  'HasO365CAS',
  'HasCASDiscovery',
  'HasPAM',
  'HasPowerBI',
  'HasPowerBIPremium',
  'HasPowerBIPro',
  'HasVisio',
  'HasProject',
  'HasNonTrial',
  'HasSubscription_AllCounted',
  'WithoutTenantAdmin',
  'HasPartnerTenants',
  'Subscriptions_HasPaidSeats',
  'HasUsGovCloudOffer',
  'Has1YearFreeDomain'],
 'input_categorical': ['CountryCode', 'Languange', 'TopParents_Industry'],
 'input_datetime': ['CreatedDate',
  'CreateDateOfFirstSubscription',
  'FirstPaidEXOStartDate',
  'FirstPaidSPOStartDate',
  'FirstPaidOD4BStartDate',
  'FirstPaidSfBStartDate',
  'FirstPaidTea

In [30]:
df['Age'].describe()

count    161725.000000
mean        676.606919
std         515.225078
min           6.000000
25%         294.000000
50%         546.000000
75%         955.000000
max        3875.000000
Name: Age, dtype: float64

In [57]:
text_cols = metadata['input_text']
df_text = df.loc[:, input_text_cols]

In [39]:
np.shape(df['Comment'].values)

(161345,)

## Test TF-IDF encode

In [122]:
def encode_one_text_col(df, col):
    tfidf = TfidfVectorizer()
    X_text = tfidf.fit_transform(df[col].values)
    return X_text, tfidf

In [123]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161345 entries, 0 to 161344
Data columns (total 2 columns):
Comment    161345 non-null object
Theme      161345 non-null object
dtypes: object(2)
memory usage: 2.5+ MB


In [124]:
X_comment, tfidf = encode_one_text_col(df_text, 'Comment')

In [125]:
type(X_comment)

scipy.sparse.csr.csr_matrix

In [126]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [190]:
df_train = df.iloc[:100, :]
df_dev = df.iloc[:10, :]
with open('raw_data/configure_wText_woPreviousUsage_NotPredectAhead.json', 'r') as f:
    metadata = json.load(f)
    
print(metadata['input_text'])

def encode_text_cols(df, metadata, text_token=[None, None]):
    text_cols = metadata['input_text']
    df_X_text = df.loc[:, text_cols]
    
    var = list(zip(text_cols, text_token))
    for i, ck in enumerate(var):
        col, tok = ck
        if tok == None:
            tok = TfidfVectorizer(max_df=0.9, min_df=10)
            X = tok.fit_transform(df_X_text[col].values)
            text_token[i] = tok
        else:
            X = tok.transform(df_X_text[col].values)
    
    return X, text_token 

# X_encoded, text_token = encode_text_cols(df_train, metadata)


['Comment', 'Theme']


In [191]:
metadata['input_text']

['Comment', 'Theme']

In [192]:
X_encoded, text_token = encode_text_cols(df_train, metadata)

In [193]:
dev_X_encoded, tk = encode_text_cols(df_dev, metadata, text_token=text_token)
len(tk)

2

In [195]:
# check the content of metadata 
metadata['input_int'][:]

['AllupSeats',
 'EXOSubscriptionsCount',
 'OD4BSubscriptionsCount',
 'SfBSubscriptionsCount',
 'TeamsSubscriptionsCount',
 'PaidCount',
 'ProjectSubscriptionsCount',
 'SPOSubscriptionsCount',
 'ActivatedSubscriptionTotalCount',
 'VisioSubscriptionsCount',
 'TrialSubscriptionsCount',
 'NonTrialSubscriptionsCount',
 'EXOEnabledUsers',
 'SPOEnabledUsers',
 'OD4BEnabledUsers',
 'SFBEnabledUsers',
 'TeamEnabledUsers',
 'YammerEnabledUsers',
 'PPDEnabledUsers',
 'KaizalaEnabledUsers',
 'AADPEnabledUsers',
 'AIPEnabledUsers',
 'AATPEnabledUsers',
 'IntuneEnabledUsers',
 'MCASEnabledUsers',
 'WDATPEnabledUsers',
 'AudioConferenceEnabledUsers',
 'PhoneSystemEnabledUsers',
 'EdiscoveryEnabledUsers',
 'ComplianceEnabledUsers ',
 'ThreatIntelligenceEnabledUsers',
 'CustomerLockboxEnabledUsers',
 'OATPEnabledUsers',
 'AADPP2EnabledUsers',
 'AIPP2EnabledUsers',
 'WindowsEnabledUsers ',
 'O365CASEnabledUsers',
 'CASDiscoveryEnabledUsers',
 'PAMEnabledUsers',
 'O365EnabledUsers',
 'EMSEnabledUsers',
 

In [184]:
int_cols = df.select_dtypes(include='float').columns.tolist()

In [196]:
int_cols[:]

['PaidAudioConferenceSeats',
 'PaidOATPSeats',
 'PaidAADPP2Seats',
 'PaidWindowsSeats',
 'PaidO365CASSeats',
 'PaidCASDiscoverySeats',
 'PaidPAMSeats',
 'PaidPowerBISeats',
 'PaidPowerBIPremiumSeats',
 'PaidPowerBIProSeats',
 'PaidEMSSeats',
 'PaidM365Seats',
 'PaidOfficeSeats',
 'Age',
 'AU_exchange_04',
 'AU_sharepoint_04',
 'AU_skype_04',
 'AU_teams_04',
 'AU_od4b_04',
 'AU_onenote_04',
 'AU_word_04',
 'AU_excel_04',
 'AU_powerpoint_04',
 'AU_outlook_04',
 'AU_oatp_04',
 'AU_odsp_04',
 'AU_eslt_04',
 'AU_officeclient_04',
 'AU_exchange_05',
 'AU_sharepoint_05',
 'AU_skype_05',
 'AU_teams_05',
 'AU_od4b_05',
 'AU_onenote_05',
 'AU_word_05',
 'AU_excel_05',
 'AU_powerpoint_05',
 'AU_outlook_05',
 'AU_oatp_05',
 'AU_odsp_05',
 'AU_eslt_05',
 'AU_officeclient_05',
 'AU_exchange',
 'AU_sharepoint',
 'AU_skype',
 'AU_teams',
 'AU_od4b',
 'AU_onenote',
 'AU_word',
 'AU_excel',
 'AU_powerpoint',
 'AU_outlook',
 'AU_oatp',
 'AU_odsp',
 'AU_eslt',
 'AU_officeclient',
 'AR_exchange_04',
 'AR_s

In [207]:
# ## figure out what's the problem is:
# df_fake = df.drop('Age', axis=1).iloc[:100, :]
# df_X_float = df_fake.loc[:, int_cols]
# df_X_float['Age']

In [77]:
df_text.head()
df_text.shape

(161345, 2)

In [62]:
tfidf.vocabulary_
rev_vocab = {}
for k, v in tfidf.vocabulary_.items():
    rev_vocab[v] = k

## Take a close look at the verbatim

In [106]:
from collections import Counter

tokenizer = tfidf.build_tokenizer()
wf_dict = {}

for comment in df_text.Comment:
    tokens = tokenizer(comment)
    for tk in tokens:
        if tk in wf_dict:
            wf_dict[tk] += 1
        else:
            wf_dict[tk] = 1

In [115]:
for comment in df_text.Comment:
    if 'OneDrive' in comment:
        print('=' * 100)
        print(comment)
        print()

Great help. Aswat took plenty of time to explain everything and help me understand the issue behind my apparent user interface problem. As it seems the new interface was a MS update and not something I have accidentally broken or miss-configured . In addition to that, I got some great tips on how to effectievly use OneDrive for Business. Thanks!

It is not possible to change ownership of shared documents! This is something that need to be solved by Microsoft as soon as possible. We found a work around now via sharepoint, but this cost time which could have been avoided Sam has been a great help! He was able to explain to me very clear what to do and how to solve. It was not that easy to convince me as we had troubles with OneDrive before, but now I fully trust that it is solved! Thanks Sam! great help, had a long call, but she never gave up on helping me and trying to explain, which at the end was successful!

Above and beyond support! The technician explained clearly to me how to use 

The person who helped me just now was very helpful getting access to the Admin locations for OneDrive. It was hidden in Apps versus under Tools. Before the personal help, we struggled to get a phone number to call Microsoft (separate online search versus in the website for OneDrive). Also the online chat was not useful at all. It took several attempts at asking questions and way off responses before being asked for a live person and even then we struggled since we needed an account first. We just wanted help. But it took a long time to get help with several iterations.

Sumit was very helpful on explaining how retention policies on Sharepoint and OneDrive applies only to deleted content and to to entire library.

Patient, and wonderfully helpful! Support Representative (Lisa) was very good - my dissatisfaction is due to there being no mechanism for consolidating/restoring files from a user's OneDrive to a place of my choosing (be it my local hard drive or the person's supervisor's OneD

In [108]:
len([p for p in wf_dict.items()])

88735

In [109]:
len([p for p in wf_dict.items() if p[1] > 5])

16232

In [119]:
wf_dict['OneDrive']

1015

In [120]:
sorted([p for p in wf_dict.items() if p[1] > 5], key=lambda x: x[1], reverse=True)[100:200]

[('questions', 5542),
 ('how', 5500),
 ('could', 5477),
 ('no', 5459),
 ('ticket', 5258),
 ('got', 5240),
 ('working', 5222),
 ('took', 5165),
 ('are', 5140),
 ('again', 5126),
 ('well', 5096),
 ('needed', 5063),
 ('him', 5031),
 ('la', 4909),
 ('assistance', 4823),
 ('fast', 4797),
 ('Office', 4790),
 ('fix', 4777),
 ('We', 4620),
 ('This', 4613),
 ('us', 4570),
 ('when', 4564),
 ('more', 4554),
 ('person', 4552),
 ('will', 4493),
 ('about', 4479),
 ('which', 4395),
 ('fixed', 4373),
 ('happy', 4344),
 ('awesome', 4297),
 ('can', 4248),
 ('one', 4196),
 ('solve', 4144),
 ('resolving', 4136),
 ('understand', 4121),
 ('resolution', 4113),
 ('she', 4088),
 ('easy', 4074),
 ('appreciate', 4069),
 ('everything', 4049),
 ('efficient', 4036),
 ('if', 3962),
 ('agent', 3953),
 ('que', 3927),
 ('received', 3911),
 ('phone', 3904),
 ('best', 3861),
 ('her', 3818),
 ('clear', 3804),
 ('el', 3756),
 ('worked', 3716),
 ('like', 3716),
 ('My', 3714),
 ('helping', 3633),
 ('polite', 3604),
 ('Servic

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]

vectorizer = TfidfVectorizer(stop_words='english')

X = vectorizer.fit_transform(corpus)
X_words = tfidf.inverse_transform(X) ## this will give you words instead of tfidf where tfidf > 0

tokenizer = vectorizer.build_tokenizer() ## return tokenizer function used in tfidfvectorizer

for idx, words in enumerate(X_words):
    for word in words:
        count = tokenizer(corpus[idx]).count(word)
        print(idx, word, count)

0 0000 0
0 00 0
0 000 0
1 000000 0
1 0000000 0
1 0000 0


In [66]:
vectorizer.vocabulary_

{'ate': 0, 'dog': 1, 'sandwich': 2, 'transfigured': 3, 'wizard': 4}

In [52]:
# Cannot use scaler again, it is already normalized
# scaler = StandardScaler()
# X_comment_sca = scaler.fit_transform(X_comment)

In [59]:
text_token = ['a', 'b']
var = list(zip(text_cols, text_token))

In [60]:
for c, t in var:
    print(c, t)

Comment a
Theme b


# Preprocess data

1. encode categorical data
2. transfer datetime data
3. convert nan to zero, inf to one
4. encode boolean type data
5. split dataset
6. normalize data

In [15]:
cols_name = pd.Series(data=df.columns)

ar_04_beg_col_index = cols_name[cols_name == 'AR_exchange_04'].index[0]
ar_06_beg_col_index = cols_name[cols_name == 'AR_exchange_06'].index[0]
ar_06_end_col_index = cols_name[cols_name == 'AR_officeclient_06'].index[0]

wl_AR_cols = cols_name[ar_04_beg_col_index : ar_06_end_col_index+1].tolist()

In [16]:
au_06_beg_col_index = cols_name[cols_name == 'AU_exchange'].index[0]
au_06_end_col_index = cols_name[cols_name == 'AU_officeclient'].index[0]

au_06_cols = cols_name[au_06_beg_col_index : au_06_end_col_index+1].tolist()

In [17]:
output_cols = cols_name[ar_06_beg_col_index : ar_06_end_col_index+1].tolist()


In [18]:
len(output_cols)
# len(wl_AR_cols)

12

In [19]:
len(au_06_cols)

14

In [6]:
def split_dataset(df, age=360):
    df_train = df.loc[df['Age'] >= age]
    df_test = df.loc[df['Age'] < age]
    
    df_train = df_train.drop('Age', axis=1)
    df_test = df_test.drop('Age', axis=1)
    
    print('df_test shape is {}'.format(df_test.shape))
    
    ### split dev set from training set ###
    dev_size = int(df_train.shape[0] * 0.01)

    df_dev = df_train.iloc[-dev_size:,:]
    df_train = df_train.iloc[:-dev_size,:]
    print('df_dev shape is {}'.format(df_dev.shape))
    print('df_train shape is {}'.format(df_train.shape))
    
    return df_train, df_dev, df_test

In [7]:
def separate_input_output_cols(df, predict_ahead=False):
    cols_name = pd.Series(data=df.columns)

    ar_04_beg_col_index = cols_name[cols_name == 'AR_exchange_04'].index[0]
    ar_06_beg_col_index = cols_name[cols_name == 'AR_exchange_06'].index[0]
    ar_06_end_col_index = cols_name[cols_name == 'AR_officeclient_06'].index[0]
    
    au_06_beg_col_index = cols_name[cols_name == 'AU_exchange'].index[0]
    au_06_end_col_index = cols_name[cols_name == 'AU_officeclient'].index[0]
    
    au_06_cols = cols_name[au_06_beg_col_index : au_06_end_col_index+1].tolist()

    wl_AR_cols = cols_name[ar_04_beg_col_index:ar_06_end_col_index+1].tolist()

    output_cols = cols_name[ar_06_beg_col_index:ar_06_end_col_index+1].tolist()

    df_y = df.loc[:, output_cols]
    print('outputs shape is {}'.format(df_y.shape))

    ### use profile info only, exclude the usage of previous months ###
    df_X = df.drop(columns=wl_AR_cols) 
    
    ### if we predict the usage at the beginning of the month, we should ###
    ### exclude the active users' number for that month (June).          ###
    if predict_ahead:
        df_X = df_X.drop(columns=au_06_cols) 
    
    print('inputs shape is {}'.format(df_X.shape))

    return df_X, df_y

In [8]:
def process_object_cols_for_inputs(df):
    ### only selected the StartDate for outputs workloads ###
    cols_datetime = ['CreatedDate', 'CreateDateOfFirstSubscription','FirstPaidEXOStartDate',
       'FirstPaidSPOStartDate', 'FirstPaidOD4BStartDate',
       'FirstPaidSfBStartDate', #'FirstPaidYammerStartDate',
       'FirstPaidTeamsStartDate', 'FirstPaidProPlusStartDate',
       #'FirstPaidAADPStartDate', 'FirstPaidAIPStartDate',
       #'FirstPaidAATPStartDate', 'FirstPaidIntuneStartDate',
       #'FirstPaidMCASStartDate', 'FirstPaidO365E5SkuStartDate',
       #'FirstPaidM365E5SkuStartDate', 'FirstPaidEMSE5SkuStartDate'
                    ]
    df_datetime = df.loc[:, cols_datetime]
    
    ### drop some redundant columns ###
    cols_cat = ['CountryCode', 'Languange', #'DataCenterInstance', 'DataCenterModel',
       #'SignupLocationInfo_Country', #'SignupLocationInfo_CountryCode',
       #'SignupLocationInfo_Region', 'TopParents_AreaName',
       #'TopParents_CountryCode', #'TopParents_BigAreaName', 
       'TopParents_Industry', #'TopParents_RegionName',
       #'TopParents_SegmentGroup', #'TopParents_SubRegionName',
       #'TopParents_VerticalName'
               ]
    df_cat = df.loc[:, cols_cat]
    
    df_tenantid = df.loc[:,'TenantId']
    
    return df_tenantid, df_cat, df_datetime

In [9]:
def encode_datetime(df_datetime):
    """Encode the datetime inputs from '2/5/2014 5:31:19 AM' format
        to a numerical number of UTC format.
        
    Args:
      df_datetime: a DataFrame that only stores the datetime inputs.
        
    Returns:
      X_datetime: a numpy array that contains the encoded datetime inputs.
      datetime_cols: a list that contains the datetime colunms name.
      
    """
    
    cols = df_datetime.columns
    for i in cols:
        df_datetime[i] = pd.to_datetime(df_datetime[i], utc=True,
                                        errors='coerce').astype(int,errors='ignore')
#     print(df_datetime.head())
        
    X_datetime = df_datetime.to_numpy()
    datetime_cols = cols.to_list()
    
    return X_datetime, datetime_cols

In [10]:
def encode_num_bool(df):
    """Encode the numerical and boolean inputs to an array.
        
    Args:
      df: a DataFrame that stores the inputs raw data.
        
    Returns:
      X_bool: a numpy array that contains the encoded boolean inputs.
      X_num: a numpy array that contains the encoded numerical inputs.
      num_cols: a list that contains the numerical columns name.
      bool_cols: a list that contains the boolean columns name.
    """
    
    X_num = df.select_dtypes(include=['float','int'])
    X_bool = df.select_dtypes(include='bool')

    num_cols = X_num.columns.to_list()
    bool_cols = X_bool.columns.to_list()
    
    X_bool = X_bool.astype(int).to_numpy()
    X_num = X_num.to_numpy()
    return X_bool, X_num, num_cols, bool_cols

In [16]:
def encode_training_inputs(df_X):
    """Encode the inputs for training set.
        
    Args:
      df_X: a DataFrame that stores the inputs raw data.
        
    Returns:
      X_scal: a numpy array that contains the encoded and normalized inputs.
      dv: a DictVectorizer that is trained on the training set.
      scaler: a StandardScaler that is trained on the training set.
      cols_name: a list that contains all of the inputs features after encoding.
      df_X_id: a DataFrame that contains the TenantId for training set.
    """
    
    
    print('Starting to encode training inputs:')
    X_bool, X_num, num_cols, bool_cols = encode_num_bool(df_X)
    
    df_X_id, df_X_cat, df_X_datetime = process_object_cols_for_inputs(df_X)
    
    X_datetime, datetime_cols = encode_datetime(df_X_datetime)
    
#     id_file_name = df_X_id.name + '_train.csv'
#     df_X_id.to_csv(id_file_name, header=False)
    
    ### encode the categorical columns ###
    X_cat_dict = df_X_cat.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_cat_encoded = dv.fit_transform(X_cat_dict)
    vocab = dv.vocabulary_
    vocab_od = collections.OrderedDict(sorted(vocab.items(), key=lambda x:x[1]))
    cat_encoded_cols = list(vocab_od.keys())
#     print(X_cat_encoded)
    
    ### normalize all the inputs ###
    X_arr = np.concatenate((X_cat_encoded, X_num, X_bool, X_datetime), axis=1)
    scaler = StandardScaler()
    X_scal = scaler.fit_transform(X_arr)
#     print(X_arr.shape)
    
    cols_name = cat_encoded_cols + num_cols + bool_cols + datetime_cols

    assert len(cols_name) == X_scal.shape[1]
    
    return X_scal, dv, scaler, cols_name, df_X_id

In [17]:
def encode_dev_test_inputs(df_X, dv, scaler):
    """Encode the inputs for dev or test set.
        
    Args:
      df_X: a DataFrame that stores the inputs raw data.
      dv: a DictVectorizer that is trained on the training set.
      scaler: a StandardScaler that is trained on the training set.
        
    Returns:
      X_scal: a numpy array that contains the encoded and normalized inputs.
      df_X_id: a DataFrame that contains the TenantId for dev or test set.
    """
    
    print('Starting to encode dev or test inputs:')
    X_bool, X_num, _, _ = encode_num_bool(df_X)
    df_X_id, df_X_cat, df_X_datetime = process_object_cols_for_inputs(df_X)
    
#     id_file_name = df_X_id.name + '_' + dataset_type + '.csv'
#     df_X_id.to_csv(id_file_name, header=False)
    
    X_datetime, _ = encode_datetime(df_X_datetime)
    
    ### encode the categorical columns ###
    X_cat_dict = df_X_cat.to_dict(orient='records')
    X_cat_encoded = dv.transform(X_cat_dict)
#     print(X_cat_encoded.shape)
    
    ### normalize all the inputs ###
    X_arr = np.concatenate((X_cat_encoded, X_num, X_bool, X_datetime), axis=1)
#     print(X_arr.shape)
    
    X_scal = scaler.transform(X_arr)
    
    return X_scal, df_X_id

In [36]:
import os
a = 'model'

model_path = os.path.join('/data/home/t-chepan/projects/MS-intern-project/results', '{}.json'.format(a))

model_path

'/data/home/t-chepan/projects/MS-intern-project/results/model.json'

In [23]:
df_train, df_dev, df_test = split_dataset(df)
df_train_X, df_train_y = separate_input_output_cols(df_train)
df_dev_X, df_dev_y = separate_input_output_cols(df_dev)
df_test_X, df_test_y = separate_input_output_cols(df_test)

df_test shape is (260, 227)
df_dev shape is (7, 227)
df_train shape is (733, 227)
outputs shape is (733, 12)
inputs shape is (733, 191)
outputs shape is (7, 12)
inputs shape is (7, 191)
outputs shape is (260, 12)
inputs shape is (260, 191)


In [24]:
Xtrain, dv, scaler, cols_name, df_train_id = encode_training_inputs(df_train_X)
Xdev, df_dev_id = encode_dev_test_inputs(df_dev_X, dv, scaler)
Xtest, df_test_id = encode_dev_test_inputs(df_test_X, dv, scaler)

print('After encoding, the training size is {}'.format(Xtrain.shape))
print('After encoding, the dev size is {}'.format(Xdev.shape))
print('After encoding, the test size is {}'.format(Xtest.shape))


Starting to encode training inputs:
Starting to encode dev or test inputs:
Starting to encode dev or test inputs:
After encoding, the training size is (733, 418)
After encoding, the dev size is (7, 418)
After encoding, the test size is (260, 418)


In [25]:
ytrain = df_train_y.to_numpy()
ydev = df_dev_y.to_numpy()
ytest = df_test_y.to_numpy()
print('Outputs are ready!')

np.save('data/ytrain_fake.npy', ytrain)
np.save('data/ydev_fake.npy', ydev)
np.save('data/ytest_fake.npy', ytest)

print('Saved the outputs targets!')

Outputs are ready!
Saved the outputs targets!


In [26]:
# with open('encoded_columns.txt', 'w') as f:
#     for item in cols_name:
#         f.write("%s\n" % item)
       
# joblib.dump(dv, 'results/vectorizer.pkl')

In [27]:
np.save('data/Xtrain_fake.npy', Xtrain)
np.save('data/Xdev_fake.npy', Xdev)
np.save('data/Xtest_fake.npy', Xtest)
print('Saved the encoded inputs!')

Saved the encoded inputs!


In [32]:
a = 0

if a:
    a = 100

a

0